*Stanislav Borysov [stabo@dtu.dk], DTU Management*
# Advanced Business Analytics

## Recommender Systems

*Based on the data from MovieLens and the [notebook](https://www.kaggle.com/devvindan/recommender-systems-basics) by Daniil Barysevich*

The purpose of this exercise is to explore the basics of Recommender Systems and to give you some intuition with code examples. It covers some popular algorithms and strategies but does not get deeply into advanced techniques or evaluation metrics.

### Contents

1. Introduction
2. Non-Personalized Recommender Systems
3. Personalized Recommender Systems   
 3.1. Content-Based Filtering  
 3.2. Collaborative Filtering
``` 
      3.2.1. User-user collaborative filtering
      3.2.2. Item-item collaborative filtering
``` 
 3.3. Matrix factorization  
 3.4. Hybrid Recommender Systems
4. Playtime: Bringing all together
5. Conclusion

### 1. Introduction

(some background information and Wikipedia text)

A *recommender system* or a *recommendation system* (sometimes replacing "system" with a synonym such as a *platform* or an *engine*) is a subclass of information filtering system that seeks to predict the "rating" or "preference" a user would give to an item.

Recommender systems are utilized in a variety of areas including movies, music, news, books, research articles, search queries, social tags, and products in general. There are also recommender systems for experts, collaborators, jokes, restaurants, garments, financial services, life insurance, romantic partners (online dating), and Twitter pages.

Recommender systems typically produce a list of recommendations in one of two ways:
 - Non-personalized approach
 - Personalized approach
 
Personalized-based recommender systems also have common subtypes:
 - Collaborative filtering recommender systems
 - Content-based recommender systems
 
All the techniques mentioned above have their problems and pitfalls, which developers face creating and applying recommender systems to real-world problems. They must be taken into account while designing the system architecture and will be covered later in this work. Though the field of recommendation itself is relatively old, there are still no solutions that work perfectly for every case. Designing and evaluating a recommender system is hard, and requires a deep understanding of domain knowledge and data available, as well as constant experimenting and modification. First recommender systems appeared a long time ago in the 1990s, but the intense research started quite recently with the availability of better computational power and tremendous amounts of data coming from the internet.

One of the events that energized research in recommender systems was the Netflix Prize. From 2006 to 2009, Netflix sponsored a competition, offering a grand prize of $1,000,000 to the team that could take an offered dataset of over 100 million movie ratings and return recommendations that were 10% more accurate than those offered by the company's existing recommender system.

This competition energized the search for new and more accurate algorithms. The most accurate algorithm in 2007 used an ensemble method of 107 different algorithmic approaches, blended into a single prediction:

>Predictive accuracy is substantially improved when blending multiple predictors. Our experience is that most efforts should be concentrated in deriving substantially different approaches, rather than refining a single technique. Consequently, our solution is an ensemble of many methods.

Beside classical approaches to recommendation with techniques described above, there are a lot of different cases that require modifications or special settings:
 - Group recommender systems
 - Context-aware recommender systems
 - Risk-aware recommender systems

### 2. Non-personalized Recommendation

Though non-personalized recommenders are rarely used in modern systems by themselves, they are still very powerful in combination with other algorithms, and, sometimes, the only available option. 

How can we make a recommendation for a user that we have little or no data about? 

That's where stereotype-based recommendations can be made, and most of the times we can take into account:
 - items popularity
 - user demographic data 
 - user actions during that particular session (for example, items in online-shop basket)
 
**Mean-based recommendation:** 

One of the common approaches we can use is a mean-based recommendation. In the simplest case, we can use mean rating for the item, $\mu_i$, to recommend items with the highest rating

$$\mu_i = \frac{\sum_x r_{xi}}{N_{i}}$$

where $r_{xi}$ is the rating for the item $i$ by the user $x$ and $N_{i}$ is the number of ratings for the item.

**Associative rule recommendation:**

This approach is used to recommend items that are related to chosen one ("People who buy this also bought...") and, therefore, uses *reference item* to provide recommendations.

The association rule formula is derived from Bayes theorem:

$$P(i|j) = \frac{P(i \vee j)}{P(j)}$$

In this case, *j* is the *reference item*, and *i* is an item to be scored. We estimate probabilities by counting: $P(j)$ is the fraction of users in the system who
purchased item $j$; $P(i\vee j)$ is the fraction that purchased both $i$ and $j$.

**Example**

The following tasks cover some examples of non-personalized data analysis based on the MovieLens dataset. Please note that there are several different data files used in this notebook for different tasks. For this task, we will use the data from `data/HW1-data.csv`. Please load the data from the file.

In [ ]:
# ...

First, find the range of ratings (minimum and maximum values) in the whole dataset.

In [ ]:
# ...

The output should be the following:
```
Minimum rating: 1.0
Maximum rating: 5.0
```

Let's calculate top movies by their mean score

In [ ]:
# ...

...and rating counts for each movie:

In [ ]:
# ...

Sometimes, we do not need to know a precise rating to make a recommendation. Therefore, we can define some ratings as positive (for example, all the ratings >= 4). Print top movies by the percentage of positive marks:

In [ ]:
# ...

Let's imagine we watched the movie "Toy Story" and we want to have a list of relevant movies to watch next. We can apply association rule here. Range movies by the percentage of people who also watched Toy Story:

In [ ]:
# ...

Making recommendations above, we did not use data about the user's gender. Statistically, men and women tend to like or dislike different kinds of movies, so, to make non-personalized recommendations more precise, we can take this information into account and see the difference. Find and print mean ratings for each movie as well as global mean ratings for male and female users:

In [ ]:
# ...

Find movies that female users rate higher than male raters:

In [ ]:
# ...

### 3. Personalized Recommendation

All the personalized recommendation require a certain amount of data collected about users. Data could either be collected implicitly (products user click on, see) and explicitly (in forms of ratings, surveys, polls). Both methods are used widely and can be combined depending on the system restrictions and type of recommendations.

#### 3.1 Content-based filtering (linear model)

Content-based filtering, also referred to as cognitive filtering, recommends items based on a comparison between the content of the items and a user profile. The content of each item is represented as a set of descriptors or terms, typically the words that occur in a document. The user profile is represented with the same terms and built up by analyzing the content of items which have been seen by the user.

Several issues have to be considered when implementing a content-based filtering system. First, terms can either be assigned automatically or manually. When terms are assigned automatically a method has to be chosen that can extract these terms from items. Second, the terms have to be represented such that both the user profile and the items can be compared in a meaningful way. Third, a learning algorithm has to be chosen that can learn the user profile based on seen items and can make recommendations based on this user profile.

The greatest advantage in "linear" content-based filtering systems is that the recommendations provided can easily be interpreted to the user because we always know what "features" about a particular item made algorithm rate it higher.

The example of content-based filtering applied to news recommendation based on terms contained in a document. Import the raw data from the excel file `data/cbf.xls`. To do this, the Pandas function `pd.read_excel(file)` can be used.

In [ ]:
raw_data = pd.read_excel("data/cbf.xls")
raw_data

The file contains topics for the documents and ratings of two users. Let's create two separate dataframes for these data.

In [ ]:
docs = raw_data.loc['doc1':'doc20', 'baseball':'family']
docs

In [ ]:
user_ranks = raw_data.loc['doc1':'doc20', 'User 1':'User 2']
#user_ranks.fillna(0, inplace=True)
user_ranks

The value of 1.0 means the user liked the document, the value of -1.0 - disliked. NaN means that the user has never seen the document (and we have to predict rating).

In the simplest case, we can learn user profiles using logistic regression corresponding to the binary problem - the probability of an item being liked by the user. Let's train a separate logistic regression model for each user. Do not forget to re-assign 0 to "dislike" values instead of -1 to make the logistic regression applicable. As usual, you can use `sklearn.linear_model.LogisticRegression`.

In [ ]:
# ...

Check the coefficients of the regression. What you can tell about each user?

In [ ]:
# ...

Range the documents by the probability of being liked by the users

In [ ]:
# ...

#### 3.2 Collaborative filtering

Collaborative filtering, also referred to as social filtering, filters information by using the recommendations of other people. It is based on the idea that people who agreed in their evaluation of certain items in the past are likely to agree again in the future. A person who wants to see a movie, for example, might ask for recommendations from friends. The recommendations of some friends who have similar interests are trusted more than recommendations from others. This information is used in the decision on which movie to see.

Collaborative filtering often uses the concept of **neighborhood** (the amount of people/items we base our prediction on). Making neighborhoods too small results in not enough information for accurate prediction, and making them too big results in high computational complexity and letting noize in systems. Neighborhood size is a hyperparameter which needs to be tuned in every system. Distance between neighbors can be defined using such metrics as cosine similarity.

One of the most common problems all collaborative filtering recommender systems face - a so-called "cold start" problem, when we either:
 - do not have enough ratings for a new user to find neighbors
 - do not have enough ratings for a new item to find neighbors
 - have a completely new system without any data to make recommendations
 
In each of those cases, problems might be solved differently depending on the particular case and options available.

##### 3.2.1 User-user collaborative filtering

In user-user collaborative filtering, we provide a recommendation based on tastes of other users similar to us. The problem with that algorithm is that we need a lot of information about other people to provide correct recommendations, but the main benefits are effectiveness and ability to provide new, unexpected, and, yet, good recommendations.

In order to account for user's tendecy to give higher/lower ratings, we will use normalization again. Algorithm for providing score based on user-user collaborative filtering is defined as:

$$ r_{xi} = \mu_{x} + \frac{\Sigma_{y \in \mathbf{K}}sim(x,y)(r_{yi}-\mu_{y})}{\Sigma_{y \in \mathbf{K}}|sim(x,y)|}  $$

Where $\mu_x$ is a mean rating for a user and $sim$ can be defined in different ways, for example, as cosine similarity or correlation.

For this example, we will use the data from the `data/data.xls` file. Again, to read the file, the pandas function `pd.read_excel(file)` can be used.

In [ ]:
# ...

Plot a correlation matrix between different users. Note, that the pandas' function `corr()` ignores missing values so it does the extra job to find overlapping rating sets for you!

In [ ]:
#correlations = data.transpose().corr()
# ...

For this example, we will make predictions for user 3867. Our 'neighborhood' for a user - users with $K=5$ highest correlations. Find these users and print their ratings and correlations to user 3867.

In [ ]:
K = 5
user_id = 3867

In [ ]:
# ...

Now, find the top 5 movies to recommend to the user.

In [ ]:
# ...

The output should look like the following:
```
77: Memento (2000)                               4.777803
275: Fargo (1996)                                4.771538
807: Seven (a.k.a. Se7en) (1995)                 4.655569
194: Amelie (2001)                               4.449936
63: Twelve Monkeys (a.k.a. 12 Monkeys) (1995)    4.396449
```

##### 3.2.2 Item-item collaborative filtering

In item-item collaborative filtering, we provide a recommendation based on other items similar to us. The benefits of it, compared to user-user collaborative filtering, is that we usually need much fewer similarity computations (in most cases, there are much more users in systems than items). The most common pitfall - the system can provide very obvious recommendations.

Score provided by item-item filtering is computed using the following formula:

$$ r_{xi} = \mu_{i} + \frac{\Sigma_{j \in \mathbf{K}}sim(i,j)(r_{xj}-\mu_{j})}{\Sigma_{j \in \mathbf{K}}|sim(i,j)|}  $$

Where $\mu_i$ is a mean rating for an item and $sim$ can be defined in different ways, for example, as cosine similarity or correlation.

Let's skip the visualization part and directly do the same top 5 predictions for the same user using the item-item algorithm. Note, that now we need to find new nearest neighbors for each movie to make this prediction.

In [ ]:
K = 5
user_id = 3867

In [ ]:
#correlations = data.corr()
# ...

The output should look the following:
```
77: Memento (2000)                                             4.640297
807: Seven (a.k.a. Se7en) (1995)                               4.481565
146: Crouching Tiger Hidden Dragon (Wo hu cang long) (2000)    4.478644
153: Lost in Translation (2003)                                4.415375
194: Amelie (2001)                                             4.207079
```

In this case, the recommendations using both algorithms are similar. However, in practice, the item-item algorithm produces better recommendations as it is easier to capture the similarity between items rather than users.

#### 3.3 Matrix factorization

In matrix factorization techniques, we usually represent the rating matrix as a product of 3 other matrices.

$$R = P\Sigma Q^{T}$$

The benefits of those techniques are that they can dramatically improve system performance by reducing the necessary amount of space. Collaborative techniques can be later applied on decomposed matrices. 

`surprise` is an easy-to-use Python package for recommender systems. Please refer to their [project page](http://surpriselib.com/) and [document page](http://surprise.readthedocs.io/en/stable/index.html#) for details of installation and tutorials. Here I will use the famous [SVD algorithm](http://sifter.org/simon/journal/20061211.html). The document for this model in the Surprise page is [here](http://surprise.readthedocs.io/en/stable/matrix_factorization.html)

We will do the same exercise as in the previous subsections and try to recommend movies to User 3867.

In [ ]:
from surprise import SVD
from surprise.dataset import Reader, Dataset

To proceed, the data for the `surprise` package should be represented in the following format: 
```
userID, movieID, rating
```
Let's create a dataframe for this:

In [ ]:
data = pd.read_excel("data/data.xls")
user_ids, movie_ids, ratings = [], [], []
#
for user_id, row in data.iterrows():
    for movie_id in data.columns:
        rating = row[movie_id]
        if not np.isnan(rating):
            user_ids.append(user_id)
            movie_ids.append(movie_id)
            ratings.append(rating)
ratings_df = pd.DataFrame({'userID':user_ids, 'movieID':movie_ids, 'rating':ratings})
ratings_df = ratings_df[['userID', 'movieID', 'rating']] # correct order
ratings_df.describe()

Now, we can create a dataset in the required format for the package.

In [ ]:
# The rating scale should be specified in Reader
reader = Reader(rating_scale=(0.5, 5.0))
data_surprise = Dataset.load_from_df(ratings_df, reader)
data_train_surprise = data_surprise.build_full_trainset() # use the whole training set 

We are ready to run the SVD algorithm. Read [documentation](https://surprise.readthedocs.io/en/stable/matrix_factorization.html#surprise.prediction_algorithms.matrix_factorization.SVD) for more details.

In [ ]:
n_factors = 15
lr_all = 0.005 # default value
reg_all = 0.02 # default value
#
model = SVD(n_factors=n_factors, lr_all=lr_all, reg_all=reg_all)
model.fit(data_train_surprise)

For this example, we will make the top 5 predictions for User 3867.

In [ ]:
user_id = 3867
missing_movies_ids = data[data.isna()].loc[user_id].index
#
recommendations = []
for movie_id in missing_movies_ids:
    r = model.predict(user_id, movie_id, verbose=False).est
    recommendations.append((movie_id, r))
    #print(movie_id)
    #break
recommendations.sort(reverse=True, key=lambda x: x[1])
recommendations[:10]
#pred = model.predict(rating_train.userID[i], rating_train.placeID[i], verbose=False)
#r_pred_train[i] = pred.est

The recommendations are now quite different from the previous two algorithms. To estimate performance of the different approaches, we would need to do train/test split (or cross-validation). We will skip this comparison here but you can try to do it yourself if you are interested.

#### 3.4 Hybrid recommender systems

In hybrid recommender systems, recommendations are made usually based on scores provided by multiple recommender systems. The most common technique is to represent the final score as a linear combination of scores provided by other recommenders with according weights. 

Another option is a so-called "switch" recommender system. Given some input, the system decides, which of the available recommender engines is better to use for a recommendation in this particular situation. Such an algorithm helps to overcome problems that exist in each recommender separately.

We also can use so-called "cascade" hybrid recommenders - the system where outputs of one recommendation algorithm are used as inputs to other. 

There are dozens of ways to use hybrid recommender systems, and there is no common way of applying them to a real-world problem. Design and architecture of each of such systems depend on data available, domain field and requirements for a particular system.

### 4. Playtime: Bringing all together

Finally, let's try to build a recommender system based on SVD and evaluate its performance against the common baselines. We will use the same data as before.

In [ ]:
# Loading and preparing the dataset
data = pd.read_excel("data/data.xls")
#data
user_ids, movie_ids, ratings = [], [], []
#
for user_id, row in data.iterrows():
    #print(row)
    for movie_id in data.columns:
        #print(movie_id, user_id)
        rating = row[movie_id]
        if not np.isnan(rating):
            #print(movie_id, user_id, row[movie_id])
            user_ids.append(user_id)
            movie_ids.append(movie_id)
            ratings.append(rating)
ratings_df = pd.DataFrame({'userID':user_ids, 'movieID':movie_ids, 'rating':ratings})
ratings_df = ratings_df[['userID', 'movieID', 'rating']] # correct order
#ratings_df

#### 4.1. Data and Train/test split

We will use 70%/% 30 train/test split. The following code, which guarantees that each movie appears at least twice in the training set, was used to create this split. Everything is pre-calculated for you, so you do not need to run this code again. The train and test indices (here, I mean actual row numbers from the data table) are loaded from the files so everyone has the same data split and can directly compare results with other students.

In [ ]:
# loading the pre-computed indices
train_ind = np.loadtxt('data/data_train.csv', dtype=int)
test_ind = np.loadtxt('data/data_test.csv', dtype=int)

In [ ]:
print("number of training samples:", train_ind.shape[0])
print("number of test samples:", test_ind.shape[0])

The train and test dataframes can be created using the `iloc` method

In [ ]:
df_train = ratings_df.iloc[train_ind]
df_test = ratings_df.iloc[test_ind]

#### 4.2. Baselines

We will use 4 baseline models:
1. Global mean
2. User's mean
3. Movie's mean
4. Global + deviation of user + deviation of the movie

Estimate them on the training set and calculate RMSE on the test set. Also, make boxplots for the visualization of the predictions. The code for the global mean baseline is given as an example.

In [ ]:
def RMSE(y_true, y_pred):
    return np.linalg.norm(y_true - y_pred) / np.sqrt(len(y_true))

In [ ]:
# A plotter to make boxplot
def MakeBoxplot(y_true, y_pred, title):
    data = [y_pred[y_true == (x*0.5+0.5)] for x in range(10)]
    fig = plt.figure(figsize=(5, 5))
    plt.boxplot(data)
    min_a, max_a = 0., 5.5
    plt.xlim((min_a, max_a))
    plt.ylim((min_a, max_a))
    plt.plot([min_a, max_a * 2], [min_a, max_a], ls='--', color='gray', linewidth=1.0)
    plt.xticks(range(12), [x*0.5 for x in range(12)])
    plt.xlabel('True Rating')
    plt.ylabel('Predicted Rating')
    plt.title(title)
    plt.show()

In [ ]:
y_true = df_test['rating'].values

In [ ]:
# global mean
global_mean = df_train['rating'].mean()
print("global_mean =", global_mean)
# prediction
y_pred = []
for i, row in df_test.iterrows():
    y_pred.append(global_mean)
y_pred = np.array(y_pred)
y_pred = np.clip(y_pred, 0.5, 5.0)
# or simply y_pred = np.array([global_mean for i in range(len(y_true))])
# performance
error = RMSE(y_true, y_pred)
print("RMSE =", error)
MakeBoxplot(y_true, y_pred, 'Test Set')

In [ ]:
# user mean
# ...

In [ ]:
# movie mean
# ...

In [ ]:
# Combined model
# ...

#### 4.3. Trying to beat the baselines with SVD

Can we beat the baselines with SVD? Note, that SVD has many hyperparameters, where the number of factors (`n_factors`) and regularization strength (`reg_all`) are the most important. As is usual in machine learning, you are not allowed to tune them on the test set. Either use cross-validation on the training set or divide the training data into two sets: validation set (~30% of the training data) and actual training set used to fit parameters of the model. 

*Hint: You can use `surprise.model_selection.GridSearchCV` for the hyperparameter tuning. Check an example here: https://surprise.readthedocs.io/en/stable/getting_started.html#tune-algorithm-parameters-with-gridsearchcv*

I managed to get RMSE = 0.85, can you do better?

In [ ]:
reader = Reader(rating_scale=(0.5, 5.0))
data_surprise = Dataset.load_from_df(df_train, reader)

In [ ]:
# ...

### 5. Conclusion

Building a good recommender system is not an easy task. Although some algorithms are considered "best practices", they all have their strengths and weaknesses. Developing a recommender system requires a good understanding of domain users, data that can be collected, and purposes of our recommendation. Without knowing all the things mentioned above, it is impossible to design a good recommender system, no matter how complicated are algorithms you use. 

However, there are common techniques that are commonly used by themselves and in combination. Modern recommender systems still use collaborative filtering and content filtering techniques, although nowadays these algorithms are used in combination and with the application of such more advanced techniques as matrix factorization, neural networks, and hybrid recommender systems.

The field of recommender systems is constantly developing, providing us with new studies on context-based recommendation, risk-aware, and group recommendations, as well as research in different evaluation methods and iterative factorization techniques. There are dozens of ways to design a recommender, and choosing "the best" approach is up to people who know why and how they want to make recommendations.